In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
# !pip install snowflake-connector-python


In [1]:
from snowflake.snowpark import Session

In [4]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
#                          database=parser['Credentials']['database'],
#                          schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [5]:
session = get_session()
# session.close()
session

In [ ]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [ ]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [ ]:
!pip freeze | grep -i cloud

In [ ]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
df = registry.show_models()
df
# type(df)

In [ ]:
model_list = registry.models()
model_list

In [ ]:
m =registry.get_model("my_model")
m

In [ ]:
df_version = m.show_versions()
df_version

In [ ]:
# default_version = m.default
m.default = "V2"

In [ ]:
mv = m.default
mv

In [ ]:
#Load default vesion as python object
# clf = mv.load()

In [ ]:
data_list = [ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9,
        10,  79,  82]
data_array = np.asarray(data_list)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction.tolist()

In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(x_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

In [ ]:
mv = registry.log_model(gbm_default,
                   model_name="my_model",
                   version_name="v2",
                   conda_dependencies=["scikit-learn", "cloudpickle"],
                   comment="My awesome ML model",
#                     python_version="3.9",
#                    metrics={"score": 96},
                   sample_input_data=x_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

In [ ]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [ ]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

# Create stage in snowflake-model-registry

In [ ]:
query = """
        create or replace stage sample_model_serving_udf_stg
        directory = (enable = true)
        copy_options = (on_error='skip_file')
        """

session.sql(query)



In [ ]:
# To see the stages
session.sql("show stages").show()

# Upload model file on ML_MODELS stage 

In [6]:
#create the stage for storing the ML models
session.sql('CREATE OR REPLACE STAGE ML_MODELS').show()

SnowparkSQLException: (1304): 01b4852f-0000-64af-0000-576d06a50c5e: 090105 (22000): Cannot perform CREATE STAGE. This session does not have a current database. Call 'USE DATABASE', or use a qualified name.

In [ ]:
session.file.put(
    "/notebooks/notebooks/ml_model", "@ML_MODELS", auto_compress=False, overwrite=True
)

# 